## Setup and Imports

In [1]:
import os
from pathlib import Path

import itk
import numpy as np
import pyvista as pv

# Import from PhysioMotion4D package
from physiomotion4d import (
    ContourTools,
    SegmentChestTotalSegmentator,
    WorkflowRegisterHeartModelToPatient,
)

## Define File Paths

In [2]:
# Patient CT image (defines coordinate frame)
patient_data_dir = Path.cwd().parent / ".." / "data" / "Slicer-Heart-CT"
patient_ct_path = patient_data_dir / "patient_img.mha"
patient_ct_heart_mask_path = patient_data_dir / "patient_heart_wall_mask.nii.gz"

# Atlas template model (moving)
atlas_data_dir = Path.cwd().parent / ".." / "data" / "KCL-Heart-Model"
atlas_vtu_path = atlas_data_dir / "average_mesh.vtk"
atlas_labelmap_path = atlas_data_dir / "average_labelmap_bkg.mha"

pca_data_dir = Path.cwd().parent / ".." / "data" / "KCL-Heart-Model" / "pca"
pca_json_path = pca_data_dir / "pca.json"
pca_group_key = "All"
pca_n_modes = 10

# Output directory
output_dir = Path.cwd() / "results"

os.makedirs(output_dir, exist_ok=True)

In [3]:
patient_image = itk.imread(str(patient_ct_path))
itk.imwrite(patient_image, str(output_dir / "patient_image.mha"), compression=True)

In [4]:
if False:
    segmentator = SegmentChestTotalSegmentator()
    segmentator.contrast_threshold = 500
    patient_segmentation_data = segmentator.segment(
        patient_image, contrast_enhanced_study=False
    )
    labelmap = patient_segmentation_data["labelmap"]
    lung_mask = patient_segmentation_data["lung"]
    heart_mask = patient_segmentation_data["heart"]
    major_vessels_mask = patient_segmentation_data["major_vessels"]
    bone_mask = patient_segmentation_data["bone"]
    soft_tissue_mask = patient_segmentation_data["soft_tissue"]
    other_mask = patient_segmentation_data["other"]
    contrast_mask = patient_segmentation_data["contrast"]

    itk.imwrite(labelmap, str(output_dir / "patient_labelmap.mha"), compression=True)

    heart_arr = itk.GetArrayFromImage(heart_mask)
    # contrast_arr = itk.GetArrayFromImage(contrast_mask)
    mask_arr = (heart_arr > 0).astype(
        np.uint8
    )  # ((heart_arr + contrast_arr) > 0).astype(np.uint8)
    patient_mask = itk.GetImageFromArray(mask_arr)
    patient_mask.CopyInformation(patient_image)

    itk.imwrite(
        patient_mask, str(output_dir / "patient_heart_mask_draft.mha"), compression=True
    )

    # hand edit fixed_mask to make patient_heart_wall_mask.nii.gz that is saved in patient_data_dir
else:
    patient_mask = itk.imread(str(patient_ct_heart_mask_path))

In [5]:
flip0 = np.array(patient_mask.GetDirection())[0, 0] < 0
flip1 = np.array(patient_mask.GetDirection())[1, 1] < 0
flip2 = np.array(patient_mask.GetDirection())[2, 2] < 0
if flip0 or flip1 or flip2:
    print("Flipping patient image...")
    print(flip0, flip1, flip2)
    flip_filter = itk.FlipImageFilter.New(Input=patient_image)
    flip_filter.SetFlipAxes([int(flip0), int(flip1), int(flip2)])
    flip_filter.SetFlipAboutOrigin(True)
    flip_filter.Update()
    patient_image = flip_filter.GetOutput()
    id_mat = itk.Matrix[itk.D, 3, 3]()
    id_mat.SetIdentity()
    patient_image.SetDirection(id_mat)
    itk.imwrite(patient_image, str(output_dir / "patient_image.mha"), compression=True)
    print("Flipping patient mask image...")
    flip_filter = itk.FlipImageFilter.New(Input=patient_mask)
    flip_filter.SetFlipAxes([int(flip0), int(flip1), int(flip2)])
    flip_filter.SetFlipAboutOrigin(True)
    flip_filter.Update()
    patient_mask = flip_filter.GetOutput()
    patient_mask.SetDirection(id_mat)
    itk.imwrite(patient_mask, str(output_dir / "patient_mask.mha"), compression=True)

__array__ implementation doesn't accept a copy keyword, so passing copy=False failed. __array__ must implement 'dtype' and 'copy' keyword arguments. To learn more, see the migration guide https://numpy.org/devdocs/numpy_2_0_migration_guide.html#adapting-to-changes-in-the-copy-keyword
__array__ implementation doesn't accept a copy keyword, so passing copy=False failed. __array__ must implement 'dtype' and 'copy' keyword arguments. To learn more, see the migration guide https://numpy.org/devdocs/numpy_2_0_migration_guide.html#adapting-to-changes-in-the-copy-keyword
__array__ implementation doesn't accept a copy keyword, so passing copy=False failed. __array__ must implement 'dtype' and 'copy' keyword arguments. To learn more, see the migration guide https://numpy.org/devdocs/numpy_2_0_migration_guide.html#adapting-to-changes-in-the-copy-keyword


Flipping patient image...
True True False


Flipping patient mask image...


In [6]:
patient_model = ContourTools().extract_contours(patient_mask)
patient_model.save(str(output_dir / "patient_mesh.vtp"))
patient_model = pv.read(str(output_dir / "patient_mesh.vtp"))

template_model = pv.read(str(atlas_vtu_path))
template_model_surface = template_model.extract_surface()
template_model_surface.save(str(output_dir / "model_surface.vtp"))
template_model_surface = pv.read(str(output_dir / "model_surface.vtp"))
template_labelmap = itk.imread(str(atlas_labelmap_path))

In [7]:
registrar = WorkflowRegisterHeartModelToPatient(
    template_model=template_model,
    template_labelmap=template_labelmap,
    template_labelmap_heart_muscle_ids=[1],
    template_labelmap_chamber_ids=[2, 3, 4, 5],
    template_labelmap_background_ids=[6],
    patient_image=patient_image,
    patient_models=[patient_model],
    pca_json_filename=pca_json_path,
    pca_group_key=pca_group_key,
    pca_number_of_modes=pca_n_modes,
)

registrar.set_mask_dilation_mm(0)
registrar.set_roi_dilation_mm(25)

patient_image = registrar.patient_image
itk.imwrite(
    patient_image, str(output_dir / "patient_image_preprocessed.mha"), compression=True
)

In [8]:
# Rough alignment using ICP
icp_results = registrar.register_model_to_model_icp()
icp_inverse_point_transform = icp_results["inverse_point_transform"]
icp_forward_point_transform = icp_results["forward_point_transform"]
icp_model_surface = icp_results["registered_template_model_surface"]
icp_labelmap = icp_results["registered_template_labelmap"]

icp_model_surface.save(str(output_dir / "icp_model_surface.vtp"))
itk.imwrite(icp_labelmap, str(output_dir / "icp_labelmap.mha"), compression=True)

2026-01-25 18:16:47 INFO WorkflowRegisterHeartModelToPatient ======================================================================


2026-01-25 18:16:47 INFO WorkflowRegisterHeartModelToPatient Stage 1: ICP Alignment (RegisterModelsICP)


2026-01-25 18:16:47 INFO WorkflowRegisterHeartModelToPatient ======================================================================


2026-01-25 18:16:47 INFO RegisterModelsICPITK ============================================================


2026-01-25 18:16:47 INFO RegisterModelsICPITK Affine Alignment Optimization


...

2026-01-25 18:20:51 INFO RegisterModelsICPITK Optimization result: [-8.14300322e-02 -3.37112552e-02  5.60658146e-02  6.50784042e+02
  3.17801530e+02  9.56867714e+02  9.30311950e-01  8.66344398e-01
  8.00000000e-01 -3.00000000e-02 -2.21845401e-02  1.57768164e-02] -> 2.139976


2026-01-25 18:20:53 INFO RegisterModelsICPITK Optimization completed!


2026-01-25 18:20:53 INFO RegisterModelsICPITK Final parameters: [-8.14300322e-02 -3.37112552e-02  5.60658146e-02  6.50784042e+02
  3.17801530e+02  9.56867714e+02  9.30311950e-01  8.66344398e-01
  8.00000000e-01 -3.00000000e-02 -2.21845401e-02  1.57768164e-02]


2026-01-25 18:20:53 INFO RegisterModelsICPITK Final mean distance: 2.14


2026-01-25 18:20:53 INFO WorkflowRegisterHeartModelToPatient Stage 1 complete: ICP alignment finished.


In [9]:
pca_results = registrar.register_model_to_model_pca()
pca_coefficients = pca_results["pca_coefficients"]
pca_model_surface = pca_results["registered_template_model_surface"]
pca_labelmap = pca_results["registered_template_labelmap"]

pca_model_surface.save(str(output_dir / "pca_model_surface.vtp"))
itk.imwrite(pca_labelmap, str(output_dir / "pca_labelmap.mha"), compression=True)

2026-01-25 18:20:54 INFO WorkflowRegisterHeartModelToPatient ======================================================================


2026-01-25 18:20:54 INFO WorkflowRegisterHeartModelToPatient Stage 2: PCA-Based Registration (RegisterModelsPCA)


2026-01-25 18:20:54 INFO WorkflowRegisterHeartModelToPatient ======================================================================


2026-01-25 18:20:54 INFO Loading PCA data from SlicerSALT format...


2026-01-25 18:20:54 INFO   JSON file: C:\src\Projects\PhysioMotion\physiomotion4d\experiments\..\data\KCL-Heart-Model\pca\pca.json


...

2026-01-25 18:22:18 INFO RegisterModelsPCA Transforming points: 133793/167240 (80.0%)


2026-01-25 18:22:19 INFO RegisterModelsPCA Transforming points: 150517/167240 (90.0%)


2026-01-25 18:22:20 INFO RegisterModelsPCA Transforming points: 167240/167240 (100.0%)


2026-01-25 18:22:20 INFO RegisterModelsPCA Registered model created with 167240 points


2026-01-25 18:22:31 INFO WorkflowRegisterHeartModelToPatient Stage 2 complete: PCA registration finished.


## Mask Alignment

In [10]:
# Perform deformable registration
print("Starting deformable mask-to-mask registration...")

m2m_results = registrar.register_mask_to_mask(use_icon_refinement=False)
m2m_inverse_transform = m2m_results["inverse_transform"]
m2m_forward_transform = m2m_results["forward_transform"]
m2m_model_surface = m2m_results["registered_template_model_surface"]
m2m_labelmap = m2m_results["registered_template_labelmap"]

print("Registration complete!")

m2m_model_surface.save(str(output_dir / "m2m_model_surface.vtp"))
itk.imwrite(m2m_labelmap, str(output_dir / "m2m_labelmap.mha"), compression=True)

2026-01-25 18:22:32 INFO WorkflowRegisterHeartModelToPatient ======================================================================


2026-01-25 18:22:32 INFO WorkflowRegisterHeartModelToPatient Stage 3: Mask-to-Mask Deformable Registration


2026-01-25 18:22:32 INFO WorkflowRegisterHeartModelToPatient ======================================================================


2026-01-25 18:22:32 INFO RegisterModelsDistanceMaps ======================================================================


2026-01-25 18:22:32 INFO RegisterModelsDistanceMaps DEFORMABLE Mask-based Registration


...

antsRegistration --dimensionality 3 -r identity --output [C:\Users\saylward\AppData\Local\Temp\tmpd3xkv6h_,00000208099BB288,00000208099BB368] --transform SyN[0.2,3,0] --metric MI[00000208099BB2E8,00000208099BB2C8,1,32] --convergence [100x70x50x0,1e-6,10] --shrink-factors 8x4x2x1 --smoothing-sigmas 3x2x1x0vox -x [00000208099BB328,00000208099BB3A8] --float 1 --write-composite-transform 0 -v 1


2026-01-25 18:24:38 INFO RegisterModelsDistanceMaps Transforming moving model...


2026-01-25 18:24:40 INFO RegisterModelsDistanceMaps DEFORMABLE mask-based registration complete!


2026-01-25 18:24:40 INFO WorkflowRegisterHeartModelToPatient Stage 3 complete: Mask-to-mask registration finished.


Registration complete!


In [11]:
print("Starting deformable registration...")
print("This may take several minutes depending on GPU availability.")

m2i_results = registrar.register_labelmap_to_image()
m2i_inverse_transform = m2i_results["inverse_transform"]
m2i_forward_transform = m2i_results["forward_transform"]
m2i_surface = m2i_results["registered_template_model_surface"]
m2i_labelmap = m2i_results["registered_template_labelmap"]
print("\nRegistration complete!")

# Save registration results to output folder
m2i_surface.save(str(output_dir / "m2i_model_surface.vtp"))
itk.imwrite(m2i_labelmap, str(output_dir / "m2i_labelmap.mha"), compression=True)

2026-01-25 18:24:41 INFO WorkflowRegisterHeartModelToPatient ======================================================================


2026-01-25 18:24:41 INFO WorkflowRegisterHeartModelToPatient Stage 4: Labelmap-to-Image Refinement (Icon Registration)


2026-01-25 18:24:41 INFO WorkflowRegisterHeartModelToPatient ======================================================================


2026-01-25 18:24:41 INFO WorkflowRegisterHeartModelToPatient Auto-generating ROI masks (dilation: 25mm)...


Starting deformable registration...
This may take several minutes depending on GPU availability.


...

2026-01-25 18:24:43 INFO WorkflowRegisterHeartModelToPatient Auto-generating ROI masks (dilation: 25mm)...


2026-01-25 18:24:44 INFO WorkflowRegisterHeartModelToPatient ROI masks auto-generated successfully.


antsRegistration --dimensionality 3 -r identity --output [C:\Users\saylward\AppData\Local\Temp\tmpgshq98se,0000020828C77E68,0000020828C77D68] --transform SyN[0.2,3,0] --metric MI[0000020828C77F88,0000020828C77FE8,1,32] --convergence [100x70x50x0,1e-6,10] --shrink-factors 8x4x2x1 --smoothing-sigmas 3x2x1x0vox -x [0000020828C77D08,0000020828C77D28] --float 1 --write-composite-transform 0 -v 1


2026-01-25 18:25:14 INFO WorkflowRegisterHeartModelToPatient Stage 4 complete: Mask-to-image registration finished.



Registration complete!


In [12]:
import time


tmp_p = itk.Point[itk.D, 3]()
point = registrar.template_model.points[0]
tmp_p[0] = float(point[0])
tmp_p[1] = float(point[1])
tmp_p[2] = float(point[2])

start_time = time.time()
# Don't save the results since ICP transform is applied as a post-PCA transform
_ = registrar.icp_registrar.forward_point_transform.TransformPoint(tmp_p)
print(f"--- ICP forward transform time: {time.time() - start_time} seconds", flush=True)

start_time = time.time()
# Don't apply the post PCA transform since this is just for setup
_ = registrar.pca_registrar.transform_point(tmp_p, include_post_pca_transform=False)
print(f"--- PCA setup time: {time.time() - start_time} seconds", flush=True)
start_time = time.time()
# Apply the post PCA transform since this is the actual transform
tmp_p = registrar.pca_registrar.transform_point(tmp_p, include_post_pca_transform=True)
print(f"PCA + ICP transform time: {time.time() - start_time} seconds", flush=True)

start_time = time.time()
tmp_p = registrar.m2m_inverse_transform.TransformPoint(tmp_p)
print(f"M2M inverse transform time: {time.time() - start_time} seconds", flush=True)

start_time = time.time()
tmp_p = registrar.m2i_inverse_transform.TransformPoint(tmp_p)
print(f"M2I inverse transform time: {time.time() - start_time} seconds", flush=True)

--- ICP forward transform time: 0.0 seconds


--- PCA setup time: 0.7905173301696777 seconds


PCA + ICP transform time: 0.0 seconds


M2M inverse transform time: 0.0 seconds


M2I inverse transform time: 0.0 seconds


In [13]:
# Verify registration using the transform member function
surface_transformed = registrar.m2i_template_model_surface
surface_transformed.save(str(output_dir / "registered_template_surface.vtp"))

model_transformed = registrar.transform_model()
model_transformed.save(str(output_dir / "registered_template.vtu"))

2026-01-25 18:25:17 INFO WorkflowRegisterHeartModelToPatient Applying transforms to model...


2026-01-25 18:25:17 INFO WorkflowRegisterHeartModelToPatient Transforming model points: 1/379158 (0.0%)


2026-01-25 18:25:18 INFO WorkflowRegisterHeartModelToPatient Transforming model points: 37916/379158 (10.0%)


2026-01-25 18:25:19 INFO WorkflowRegisterHeartModelToPatient Transforming model points: 75831/379158 (20.0%)


2026-01-25 18:25:20 INFO WorkflowRegisterHeartModelToPatient Transforming model points: 113746/379158 (30.0%)


...

2026-01-25 18:25:26 INFO WorkflowRegisterHeartModelToPatient Transforming model points: 303321/379158 (80.0%)


2026-01-25 18:25:27 INFO WorkflowRegisterHeartModelToPatient Transforming model points: 341236/379158 (90.0%)


2026-01-25 18:25:28 INFO WorkflowRegisterHeartModelToPatient Transforming model points: 379151/379158 (100.0%)


2026-01-25 18:25:28 INFO WorkflowRegisterHeartModelToPatient Transforming model points: 379158/379158 (100.0%)


2026-01-25 18:25:28 INFO WorkflowRegisterHeartModelToPatient Transform application complete.


## Visualize Final Results

In [14]:
# Load meshes from registrar member variables
patient_surface = registrar.patient_model_surface
registered_surface = registrar.registered_template_model_surface
icp_surface = registrar.icp_template_model_surface
pca_surface = registrar.pca_template_model_surface
m2m_surface = registrar.m2m_template_model_surface
m2i_surface = registrar.m2i_template_model_surface

# Create side-by-side comparison
plotter = pv.Plotter(shape=(1, 2))

# After rough alignment
plotter.subplot(0, 0)
plotter.add_mesh(patient_surface, color="red", opacity=0.5, label="Patient")
plotter.add_mesh(pca_surface, color="green", opacity=1.0, label="After ICP")
plotter.add_title("PCA Alignment")

# After deformable registration
plotter.subplot(0, 1)
plotter.add_mesh(patient_surface, color="red", opacity=0.5, label="Patient")
plotter.add_mesh(m2i_surface, color="blue", opacity=1.0, label="Registered")
plotter.add_title("Final Registration")

plotter.link_views()
plotter.show()

Widget(value='<iframe src="http://localhost:59182/index.html?ui=P_0x207d51cb8b0_0&reconnect=auto" class="pyvis…

## Visualize Deformation Magnitude

In [15]:
# The transformed mesh has deformation magnitude stored as point data
if "DeformationMagnitude" in registered_surface.point_data:
    plotter = pv.Plotter()
    plotter.add_mesh(
        registered_surface,
        scalars="DeformationMagnitude",
        cmap="jet",
        show_scalar_bar=True,
        scalar_bar_args={"title": "Deformation (mm)"},
    )
    plotter.add_title("Deformation Magnitude")
    plotter.show()

    # Print statistics
    deformation = registered_surface["DeformationMagnitude"]
    print("Deformation statistics:")
    print(f"  Min: {deformation.min():.2f} mm")
    print(f"  Max: {deformation.max():.2f} mm")
    print(f"  Mean: {deformation.mean():.2f} mm")
    print(f"  Std: {deformation.std():.2f} mm")
else:
    print("DeformationMagnitude not found in mesh point data")

Widget(value='<iframe src="http://localhost:59182/index.html?ui=P_0x207abcb7340_1&reconnect=auto" class="pyvis…

Deformation statistics:
  Min: 0.19 mm
  Max: 16.51 mm
  Mean: 6.36 mm
  Std: 2.71 mm
